In [5]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, MaxAbsScaler
from sklearn.feature_selection import mutual_info_classif, chi2, f_classif
from sklearn.neighbors import LocalOutlierFactor
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp
import os
import json
from scipy import stats
from sklearn.neighbors import KernelDensity

# Suppress TensorFlow logs for cleaner output
tf.get_logger().setLevel('ERROR')

### Helper Functions

#### Residual Block for Autoencoder
def residual_block(x, units):
    """Creates a residual block with two dense layers to improve gradient flow."""
    shortcut = x
    x = layers.Dense(units)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(units)(x)
    x = layers.BatchNormalization()(x)
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

#### Focal Loss for Classification
def focal_loss(gamma=2.0, alpha=None):
    """Focal loss function to address class imbalance in multi-class classification."""
    alpha = tf.constant(alpha, dtype=tf.float32) if alpha is not None else None
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        p_t = tf.gather(y_pred, y_true, batch_dims=1)
        ce = -tf.math.log(p_t)
        alpha_t = tf.gather(alpha, y_true) if alpha is not None else 1.0
        focal_weight = alpha_t * tf.math.pow((1 - p_t), gamma)
        return focal_weight * ce
    return loss

#### Data Augmentation
def augment_sequence(sequence, noise_level=0.01):
    """Applies Gaussian noise to a sequence for data augmentation."""
    noise = np.random.normal(0, noise_level, sequence.shape)
    return sequence + noise

#### Custom Sequence Generator
class SequenceGenerator(tf.keras.utils.Sequence):
    """Custom sequence generator for efficient batch processing with optional augmentation."""
    def __init__(self, X, y, batch_size, augment=False):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.augment = augment

    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch_size))

    def __getitem__(self, idx):
        batch_X = self.X[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        if self.augment:
            batch_X = np.array([augment_sequence(seq) for seq in batch_X])
        return batch_X, batch_y

### Layer 1: Autoencoder for Anomaly Detection and Feature Extraction
class EnhancedAdaptiveNIDS:
    def __init__(self, input_dim, latent_dim=64, learning_rate=1e-4):
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.learning_rate = learning_rate
        self.model = self._build_autoencoder()
        self.kde = None
        self.threshold = None

    def _build_autoencoder(self):
        """Builds an Autoencoder with residual connections for stability."""
        inputs = layers.Input(shape=(self.input_dim,))
        x = layers.Dense(128, activation='relu')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        x = residual_block(x, 128)
        x = layers.Dense(64, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        x = residual_block(x, 64)
        encoded = layers.Dense(self.latent_dim, activation='relu')(x)
        x = layers.Dense(64, activation='relu')(encoded)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        x = residual_block(x, 64)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
        x = residual_block(x, 128)
        decoded = layers.Dense(self.input_dim, activation='linear')(x)
        autoencoder = keras.Model(inputs=inputs, outputs=decoded)
        autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate), loss='huber')
        return autoencoder

    def train(self, X_train, X_val, epochs=50, batch_size=64):
        """Trains the Autoencoder and sets a dynamic anomaly threshold."""
        early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
        history = self.model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size,
                                 validation_data=(X_val, X_val), callbacks=[early_stopping, reduce_lr], verbose=1)
        self._set_dynamic_threshold(X_train)
        return history

    def _set_dynamic_threshold(self, X_data):
        """Sets a dynamic threshold using Kernel Density Estimation."""
        reconstructed = self.model.predict(X_data)
        errors = np.mean(np.abs(X_data - reconstructed), axis=1)
        self.kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(errors.reshape(-1, 1))
        log_dens = self.kde.score_samples(errors.reshape(-1, 1))
        self.threshold = np.percentile(-log_dens, 95)  # 95th percentile as anomaly threshold
        print(f"Dynamic threshold set to: {self.threshold}")

    def detect_anomalies(self, X_data):
        """Detects anomalies based on reconstruction errors."""
        if self.kde is None or self.threshold is None:
            raise ValueError("Model hasn't been trained yet. Call train() first.")
        reconstructed = self.model.predict(X_data)
        errors = np.mean(np.abs(X_data - reconstructed), axis=1)
        log_dens = self.kde.score_samples(errors.reshape(-1, 1))
        anomaly_indices = np.where(-log_dens > self.threshold)[0]
        confidence = -log_dens / np.max(-log_dens) if len(log_dens) > 0 else np.array([])
        return X_data[anomaly_indices], anomaly_indices, errors, confidence

    def get_encoded_features(self, X_data):
        """Extracts encoded features from the Autoencoder's latent layer."""
        encoder = keras.Model(inputs=self.model.input, outputs=self.model.layers[8].output)  # Latent layer index
        return encoder.predict(X_data)

    def get_feature_importance(self, X_data, anomaly_indices):
        """Computes feature importance based on reconstruction errors of anomalies."""
        reconstructed = self.model.predict(X_data)
        errors = np.abs(X_data - reconstructed)
        anomaly_errors = errors[anomaly_indices]
        feature_importance = np.mean(anomaly_errors, axis=0)
        return feature_importance

### Layer 2: CNN-BiLSTM for Attack Classification
class AdaptiveNIDSLayer2:
    def __init__(self, input_dim, num_classes, seq_length=10):
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.seq_length = seq_length
        self.temperature = tf.Variable(1.0, trainable=True)
        self.model = self._build_model()
        self.class_weights = None

    def _build_model(self):
        """Builds a CNN-BiLSTM model with attention mechanism."""
        inputs = layers.Input(shape=(self.seq_length, self.input_dim))
        x = layers.Conv1D(64, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling1D(2)(x)
        x = layers.Conv1D(128, 3, activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4))(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling1D(2)(x)
        shortcut = layers.Conv1D(128, 1, kernel_regularizer=regularizers.l2(1e-4))(inputs)
        shortcut = layers.BatchNormalization()(shortcut)
        shortcut = layers.MaxPooling1D(4)(shortcut)
        x = layers.add([x, shortcut])
        x = layers.Activation('relu')(x)
        x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, kernel_regularizer=regularizers.l2(1e-4)))(x)
        x = layers.Dropout(0.25)(x)
        x = layers.Bidirectional(layers.LSTM(32, return_sequences=True, kernel_regularizer=regularizers.l2(1e-4)))(x)
        # Attention mechanism
        attention_scores = layers.Dense(1, activation=None)(x)
        attention_weights = layers.Softmax(axis=1)(attention_scores)
        context_vector = layers.Lambda(lambda inputs: tf.reduce_sum(inputs[0] * inputs[1], axis=1))([x, attention_weights])
        dense1 = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(context_vector)
        dense1 = layers.BatchNormalization()(dense1)
        dense1 = layers.Dropout(0.25)(dense1)
        logits = layers.Dense(self.num_classes)(dense1)
        scaled_logits = layers.Lambda(lambda x: x / self.temperature)(logits)
        outputs = layers.Activation('softmax')(scaled_logits)
        return keras.Model(inputs=inputs, outputs=outputs)

    def compute_class_weights(self, y_train):
        """Computes class weights to handle imbalance."""
        y_train_int = y_train.astype(int)
        unique_classes = np.unique(y_train_int)
        class_counts = np.bincount(y_train_int)
        total = len(y_train_int)
        self.class_weights = {i: total / (len(unique_classes) * count) for i, count in enumerate(class_counts)}
        print("Class weights:", self.class_weights)
        return self.class_weights

    def train(self, X_train, y_train, X_val=None, y_val=None, epochs=50, batch_size=64):
        """Trains the CNN-BiLSTM model with focal loss and gradient clipping."""
        if self.class_weights is None:
            self.compute_class_weights(y_train)
        alpha = np.array([self.class_weights.get(i, 1.0) for i in range(self.num_classes)])
        self.model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-3, clipvalue=1.0),
            loss=focal_loss(gamma=2.0, alpha=alpha),
            metrics=['accuracy']
        )
        callbacks = [
            keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, min_lr=1e-7),
            keras.callbacks.ModelCheckpoint('best_layer2_model.h5', save_best_only=True, monitor='val_accuracy', mode='max'),
            keras.callbacks.TensorBoard(log_dir=f'./logs/layer2_{time.strftime("%Y%m%d-%H%M%S")}', histogram_freq=1)
        ]
        train_generator = SequenceGenerator(X_train, y_train, batch_size, augment=True)
        val_generator = SequenceGenerator(X_val, y_val, batch_size, augment=False) if X_val is not None else None
        if val_generator:
            history = self.model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=callbacks)
        else:
            history = self.model.fit(train_generator, epochs=epochs, validation_split=0.2, callbacks=callbacks)
        return history

    def evaluate(self, X_test, y_test):
        """Evaluates the model and generates performance metrics."""
        test_loss, test_accuracy = self.model.evaluate(X_test, y_test, verbose=1)
        print(f"Test accuracy: {test_accuracy:.4f}")
        print(f"Test loss: {test_loss:.4f}")
        y_pred_probs = self.model.predict(X_test)
        y_pred = np.argmax(y_pred_probs, axis=1)
        report = classification_report(y_test, y_pred, output_dict=True)
        print("Classification Report:")
        print(classification_report(y_test, y_pred))
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig('confusion_matrix.png')
        plt.close()
        return report, y_pred, y_pred_probs

    def save_model(self, filepath):
        """Saves the trained model to a file."""
        self.model.save(filepath)
        print(f"Model saved to {filepath}")

### Data Preprocessing
def preprocess_data(file_path, test_size=0.2, random_state=42, use_adasyn=True, top_k_features=50):
    """Preprocesses data with ensemble feature selection, scaling, and outlier removal."""
    df = pd.read_csv(file_path)
    if df.empty or 'Attack_label' not in df.columns:
        raise ValueError("Dataset is empty or missing 'Attack_label' column.")
    X = df.drop(['Attack_label'], axis=1)
    y = df['Attack_label'].astype(int)
    print("Class distribution before preprocessing:", y.value_counts(normalize=True) * 100)

    # Ensemble feature selection
    if np.any(X < 0):
        score_funcs = [mutual_info_classif, f_classif]
    else:
        score_funcs = [mutual_info_classif, chi2, f_classif]
    feature_scores = []
    for score_func in score_funcs:
        try:
            if score_func == chi2:
                scores, _ = score_func(X, y)
            else:
                scores = score_func(X, y)
            scores = np.nan_to_num(scores, nan=0.0)
            feature_scores.append(scores)
        except Exception as e:
            print(f"Error in {score_func.__name__}: {e}")
            continue
    if len(feature_scores) == 0:
        raise ValueError("No valid feature scores computed.")
    expected_length = X.shape[1]
    ranks = [np.argsort(np.argsort(-scores)) for scores in feature_scores if len(scores) == expected_length]
    if len(ranks) == 0 or any(len(rank) != expected_length for rank in ranks):
        raise ValueError("Inconsistent feature ranking lengths.")
    average_rank = np.mean(ranks, axis=0)
    selected_indices = np.argsort(average_rank)[:min(top_k_features, X.shape[1])]
    X_selected = X.iloc[:, selected_indices]
    print(f"Selected {len(selected_indices)} features: {X.columns[selected_indices][:5].tolist()}...")

    # Scaling
    scaler = PowerTransformer(method='yeo-johnson') if np.any(np.abs(stats.skew(X_selected)) > 1) else MaxAbsScaler()
    scaler.fit(X_selected)
    X_scaled = scaler.transform(X_selected)
    joblib.dump(scaler, 'scaler.pkl')

    # Outlier removal
    lof = LocalOutlierFactor(n_neighbors=20, contamination='auto', n_jobs=min(mp.cpu_count(), 4))
    outlier_labels = lof.fit_predict(X_scaled)
    outlier_mask = outlier_labels == 1
    X_scaled = X_scaled[outlier_mask]
    y = y[outlier_mask]
    print(f"Removed {np.sum(~outlier_mask)} outliers.")

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=test_size, random_state=random_state, stratify=y if len(np.unique(y)) > 1 else None
    )
    if use_adasyn and len(np.unique(y_train)) > 1:
        adasyn = ADASYN(random_state=random_state)
        X_train, y_train = adasyn.fit_resample(X_train, y_train)
        print("Class distribution after ADASYN:", pd.Series(y_train).value_counts(normalize=True) * 100)
    return X_train, X_test, y_train, y_test, scaler, X.columns[selected_indices].tolist()

### Sequence Creation
def create_sequences(data, labels, seq_length=10, stride=2, min_seq_variance=0.01):
    """Creates sequences from data with adaptive length based on variance."""
    if len(data) < seq_length:
        raise ValueError(f"Data length ({len(data)}) < sequence length ({seq_length}).")
    effective_seq_length = seq_length if np.mean(np.var(data, axis=0)) >= min_seq_variance else max(5, seq_length // 2)
    sequences, seq_labels = [], []
    for i in range(0, len(data) - effective_seq_length + 1, stride):
        seq = data[i:i + effective_seq_length]
        sequences.append(seq)
        seq_labels.append(labels[i + effective_seq_length - 1])
    return np.array(sequences), np.array(seq_labels)

### Main Execution
if __name__ == "__main__":
    start_time = time.time()
    os.makedirs('./logs', exist_ok=True)
    os.makedirs('./models', exist_ok=True)
    np.random.seed(42)
    tf.random.set_seed(42)

    # Load and preprocess data
    dataset_path = "training_dataset.csv"  # Replace with your dataset path
    X_train, X_test, y_train, y_test, scaler, selected_features = preprocess_data(dataset_path)

    # Train Layer 1: Autoencoder
    print("\nTraining Layer 1: Autoencoder...")
    layer1 = EnhancedAdaptiveNIDS(input_dim=X_train.shape[1])
    layer1.train(X_train, X_test)
    anomalies, anomaly_indices, errors, confidence = layer1.detect_anomalies(X_test)
    print(f"Detected {len(anomaly_indices)} anomalies out of {len(X_test)} samples.")
    feature_importance = layer1.get_feature_importance(X_test, anomaly_indices)
    print("Top 5 important features for anomalies:")
    sorted_indices = np.argsort(-feature_importance)
    for i in sorted_indices[:5]:
        print(f"{selected_features[i]}: {feature_importance[i]:.4f}")
    encoded_features = layer1.get_encoded_features(anomalies)
    y_anomalies = y_test[anomaly_indices] if not isinstance(y_test, pd.Series) else y_test.iloc[anomaly_indices]
    X_layer2, y_layer2 = create_sequences(encoded_features, y_anomalies)

    # Split data for Layer 2
    X_train_l2, X_test_l2, y_train_l2, y_test_l2 = train_test_split(
        X_layer2, y_layer2, test_size=0.2, random_state=42, stratify=y_layer2 if len(np.unique(y_layer2)) > 1 else None
    )

    # Train Layer 2: CNN-BiLSTM
    print("\nTraining Layer 2: CNN-BiLSTM...")
    layer2 = AdaptiveNIDSLayer2(input_dim=X_train_l2.shape[2], num_classes=len(np.unique(y_train_l2)))
    layer2.train(X_train_l2, y_train_l2, X_test_l2, y_test_l2)
    report, y_pred, y_pred_probs = layer2.evaluate(X_test_l2, y_test_l2)

    # Save models and configuration
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    layer1.model.save(f'models/autoencoder_model_{timestamp}.h5')
    layer2.save_model(f'models/cnn_bilstm_model_{timestamp}.h5')
    config = {
        'dataset_path': dataset_path,
        'layer1': {'latent_dim': layer1.latent_dim, 'threshold': layer1.threshold},
        'layer2': {'seq_length': layer2.seq_length, 'class_weights': layer2.class_weights},
        'timestamp': timestamp
    }
    with open(f'models/config_{timestamp}.json', 'w') as f:
        json.dump(config, f, indent=4)

    # Execution time
    elapsed_time = time.time() - start_time
    print(f"Total execution time: {elapsed_time:.2f} seconds")

Class distribution before preprocessing: Attack_label
1    85.994594
0    14.005406
Name: proportion, dtype: float64


/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [ 6  9 28 29 30 31 36 41 42 43] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


Selected 44 features: ['Unnamed: 0', 'mqtt.hdrflags', 'tcp.dstport', 'mqtt.msgtype', 'mqtt.len']...
Removed 1597 outliers.
Class distribution after ADASYN: Attack_label
0    50.024538
1    49.975462
Name: proportion, dtype: float64

Training Layer 1: Autoencoder...
Epoch 1/50
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 0.3976 - val_loss: 0.1431 - learning_rate: 1.0000e-04
Epoch 2/50
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1510 - val_loss: 0.1055 - learning_rate: 1.0000e-04
Epoch 3/50
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1139 - val_loss: 0.0771 - learning_rate: 1.0000e-04
Epoch 4/50
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0795 - val_loss: 0.0607 - learning_rate: 1.0000e-04
Epoch 5/50
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0625 - val_loss: 0.0507 - learning_rate: 1.0000e-04
Epoch 6/50
1497/1497 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.0515 - val_loss: 0.0429 - learning_rate: 1.0000e-04
Epoch 7/50
1497/1497 ━━━━━━━━━━━━━━━━━

KeyError: 9

In [3]:
df_temp = pd.read_csv('training_dataset.csv')

In [2]:
import pandas as pd

In [8]:
df_temp["Attack_label"].value_counts()

Attack_label
1.0    61401
0.0    10000
Name: count, dtype: int64